<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/vector_drug_ResNet50%26YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

def unzip_file(zip_file, extract_folder):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print(f"ไฟล์ {zip_file} ถูก unzip ไปที่ {extract_folder} แล้ว")

# ตัวอย่างการใช้งาน
zip_file = '/content/drive/MyDrive/drug/drug50cls_640.zip'  # เปลี่ยนเป็นชื่อไฟล์ที่ต้องการ unzip
extract_folder = '/content/drug50cls_640'  # เปลี่ยนเป็นโฟลเดอร์ที่ต้องการที่จะ extract ไฟล์ไปที่นั่น

unzip_file(zip_file, extract_folder)

In [ ]:
!pip install tensorflow

In [ ]:
!pip install ultralytics

##ไม่ใช้YOLO

In [12]:
import zipfile

# ระบุพาธไฟล์ ZIP ที่คุณต้องการจะแตก
zip_file_path = '/content/drive/MyDrive/drug/drug50cls_640.zip'
# ระบุพาธที่ต้องการเก็บไฟล์ที่แตกแล้ว
extract_to_path = '/content/drug50cls_640'

# แตกไฟล์ ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f'ไฟล์ ZIP ถูกแตกออกที่ {extract_to_path}')

ไฟล์ ZIP ถูกแตกออกที่ /content/drug50cls_640


In [ ]:
# Test more than 1
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image  # Import PIL for image handling

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(image_path, all_embeddings):
    new_embedding = image_embedding(image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def process_images_in_folder(folder_path, all_embeddings):
    for image_file in os.listdir(folder_path):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder_path, image_file)
            print(f"Processing image: {image_file}")

            similar_classes = find_most_similar_classes(image_path, all_embeddings)

            # แสดงภาพโดยใช้ matplotlib
            img = image.load_img(image_path)
            plt.imshow(img)
            plt.axis('off')  # ปิดแกน X และ Y
            plt.show()

            print("Top 5 most similar classes:")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่ต้องการทดสอบ
new_image_folder_path = '/content/drive/MyDrive/drug/test'

# ประมวลผลรูปภาพทั้งหมดในโฟลเดอร์
process_images_in_folder(new_image_folder_path, all_embeddings)

ผิดไป 17class

##ใช้YOLO

In [ ]:
# Crop All
import os
from PIL import Image
from ultralytics import YOLO

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/model.pt')

# Define the input path (can be a single image or a folder)
path = '/content/drive/MyDrive/drug/test'

# Define the crop directory
crop_dir = '/content/CorpTest'
os.makedirs(crop_dir, exist_ok=True)

# Function to get image ID from file name
def get_image_id(file_name):
    return os.path.splitext(os.path.basename(file_name))[0]

# Function to perform object detection and cropping
def detect_and_crop(image_path, image_id):
    # Predict with the YOLO model
    results = model(image_path)

    # Parse the results
    detected = False
    for result in results:
        # Extract boxes, labels, and scores
        boxes = result.boxes.xyxy  # x1, y1, x2, y2
        labels = result.boxes.cls  # class ids
        scores = result.boxes.conf  # confidence scores

        # Convert tensor to numpy for easier processing
        boxes = boxes.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        scores = scores.cpu().detach().numpy()

        # Threshold for detection confidence
        threshold = 0.3

        # List of target labels (Replace with actual indices of your classes in the YOLO model)
        target_labels = ['Amlopine_10mg', 'Amlopine_5mg', 'Anapril_5mg', 'Betalol-10-mg', 'Betalol-40-mg',
                         'Blopress-16-mg', 'Blopress-8-mg', 'Blopress-Plus-16-mg', 'Blopress-Plus-8-mg',
                         'Caduet-10_10-mg', 'Caduet-5_10-mg', 'Daonil-5-mg', 'Diamicron-MR-60-mg',
                         'Diovan-160-mg', 'Diovan-80-mg', 'Forxiga-10-mg', 'Galvus-50-mg', 'Galvus_Met_50_1000mg',
                         'Gliclazide_80mg', 'Gliparil-2-mg', 'Glucophage-500-mg', 'Glucophage-850-mg',
                         'Glucophage-XR-1000-mg', 'Glucophage-XR-750-mg', 'Glyxambi-25_5-mg', 'Janumet-50_1000-mg',
                         'Januvia-100-mg', 'Jardiance-10-mg', 'Jardiance-25-mg', 'Jardiance-Duo-12_5_1000-mg',
                         'Lanzaar-100-mg', 'Lercadip-20-mg', 'Madiplot-10-mg', 'Madiplot_20mg',
                         'Metoprolol_Stada_100mg', 'Micardis_40mg', 'Micardis_Plus_80_12_5mg', 'Minidiab_5mg',
                         'Novonorm_1mg', 'Novonorm_2mg', 'Oseni_25_15mg', 'Poli_uretic', 'Prenolol_100mg',
                         'Prenolol_25mg', 'Prenolol_50mg', 'Tanzaril_50mg', 'Utmos_15mg', 'Utmos_30mg',
                         'Xigduo_XR_10_1000mg', 'Zanidip_10mg']

        # Create a map from YOLO class index to target label
        class_map = {i: label for i, label in enumerate(target_labels)}

        # Check if any of the target labels are detected
        for i, label in enumerate(labels):
            if class_map[label] in target_labels and scores[i] >= threshold:
                score = scores[i]
                box = boxes[i]

                # Get the coordinates and dimensions of the detected object
                x1, y1, x2, y2 = box

                # Open the original image
                im = Image.open(image_path)

                # Crop the image using the detected coordinates
                im_crop = im.crop((x1, y1, x2, y2))

                # Convert image mode to RGB if it's RGBA
                if im_crop.mode == 'RGBA':
                    im_crop = im_crop.convert('RGB')

                # Save the cropped image with ID
                crop_path = os.path.join(crop_dir, f'{image_id}.jpg')
                im_crop.save(crop_path)
                print(f"Cropped image saved as '{crop_path}'.")
                detected = True
                return image_id  # Return the image ID when successfully cropped

    if not detected:
        print(f"Can't detect any target objects in image ID '{image_id}'.")
        return None  # Return None when no objects detected

# Check if path is a file or directory
if os.path.isfile(path):
    image_id = get_image_id(path)
    detect_and_crop(path, image_id)
elif os.path.isdir(path):
    # Loop through all files in the directory
    for idx, file_name in enumerate(sorted(os.listdir(path))):
        file_path = os.path.join(path, file_name)
        if os.path.isfile(file_path):
            image_id = f"image{idx + 1}"
            detect_and_crop(file_path, image_id)
else:
    print(f"Invalid path: {path}")

In [ ]:
# Test more than 1
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image  # Import PIL for image handling

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(image_path, all_embeddings):
    new_embedding = image_embedding(image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def process_images_in_folder(folder_path, all_embeddings):
    for image_file in os.listdir(folder_path):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder_path, image_file)
            print(f"Processing image: {image_file}")

            similar_classes = find_most_similar_classes(image_path, all_embeddings)

            # แสดงภาพโดยใช้ matplotlib
            img = image.load_img(image_path)
            plt.imshow(img)
            plt.axis('off')  # ปิดแกน X และ Y
            plt.show()

            print("Top 5 most similar classes:")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่ต้องการทดสอบ
new_image_folder_path = '/content/CorpTest'

# ประมวลผลรูปภาพทั้งหมดในโฟลเดอร์
process_images_in_folder(new_image_folder_path, all_embeddings)

ผิดไป 3class